# 3. Redes neuronales - De 101 a PRO

¡Bienvenidos a la tercera sesión! Tras la paliza de ayer con el backpropagation y cómo funcionan realmente las redes neuronales, hoy vamos a ver lo siguiente:

**Redes neuronales 2**
* Teoría de optimización. Gradient Descent (GD, SGD, Mini-batch GD).
* Learning rate y batch size.
* Funciones de pérdidas y funciones de activación.
* Inicialización de los pesos

## 3.1 Teoría de optimización: Gradient Descent, Stochastic Gradient Descent, Mini-batch Gradient Descent

Ayer estuvimos viendo cómo funciona el gradient descent, os acordáis, ¿verdad? Cómo olvidarlo...eh? Bueno, tranquilos porque la clase de hoy es mucho mas *light*: vamos a ver diferentes formas de utilizar el gradient descent para ganar velocidad.

Necesito que recordéis cómo funciona el descenso del gradiente: calculando el calculando el error de todas las muestras de nuestro training y luego actualizando los pesos en la dirección que indica el gradiente respecto a cada uno de ellos. Es decir, necesitamos, **para cada época, calcular todas las predicciones (etapa forward), luego los errores, y luego propagar los errores hacia atrás para ver cuanto influye cada peso en ese error y actualizarlo en consecuencia.**

Imagináos que:
* tenemos un dataset de 100.000 muestras
* que cada etapa forward tarda 2ms
* cada cálculo del error 1ms
* cada etapa de backpropagation tarda 3ms

Si hacemos el cálculo, tenemos que:

**Tiempo por época** $= (2ms+1ms+3ms)·100.000$ muestras $= 600.000ms = 600$ segundos $=$ **10 minutos**

De normal, una red puede requerir cientos, si no miles de épocas, para conseguir una convergencia adecuada. Pongamos que necesitamos 100 épocas, que es un número bajo. ¿Cuánto tardaría en total en entrenarse la red?

**Tiempo total de entrenamiento** $=10$ minutos $· 100$ épocas $= 1.000$ minutos $=$ **16'6 horas !!!**

No parece muy cómodo tener que esperar más de 16 horas para ver los resultados de una red, verdad? Y hemos sido buenos suponiendo que teníamos solo 100.000 muestras. ImageNet, por ejemplo, consta de 1.2 millones de imágenes, lo que nos costaría 2h por época, o lo que es lo mismo, 8.3 días, **más de una semana para ver el comportamiento de una red.** 

Veis la necesidad, ¿no? ¿Se os ocurre alguna forma de disminuir este tiempo?

Una forma de reducir drásticamente el tiempo necesario sería utilizar una sola muestra escogida aleatoriamente cada vez que quisieramos actualizar los pesos. De esta forma, para actualizar los pesos simplemente tendríamos que calcular las predicciones, errores y backpropagation de una muestra. Esto reduciría nuestro tiempo total a:

**Tiempo total de entrenamiento** $=(2ms+1ms+3ms)·1$ muestra $·100$ épocas $= 600ms = $ **0,6s**

### Eureka! Qué maravilla! Acabamos de arreglar el mundo!

A este método se le conoce como **Stochastic Gradient Descent**, y estoy seguro de que todos vosotros os habéis dado cuenta de que tiene una **desventaja MUY IMPORTANTE**. ¿Quién me la dice?

Os voy a dar una pista:

### ¿Cuál creéis que es el camino seguido por el Gradient Descent, y cual el seguido por el Stochastic Gradient Descent?
<img src="https://image.ibb.co/hiK2BT/mountain_gd_sgd_mbsgd_hidden.png" alt="mountain_gd_sgd_mbsgd_hidden" border="0">
<!--
<img src="https://image.ibb.co/n2RFWT/mountain_gd_sgd_mbsgd.png" alt="mountain_gd_sgd_mbsgd" border="0">
<!-- <img src="https://image.ibb.co/jZHNBT/contours_gd_sgd_mbsgd.png" alt="contours_gd_sgd_mbsgd" border="0"> -->

El camino rojo es el que sigue el gradient descent, que al calcular el gradiente usando todas las muestras del dataset consigue unas actualizaciones coherentes siempre en la dirección que permite minimizar el error.

Y si! El camino magenta es el seguido por el SGD. ¿Qué es lo que está pasando? Que cada actualización de los pesos se hace para minimizar el error teniendo en cuenta solo una muestra, así que lo que minimizamos es el error para esa muestra en particular. Por eso tiene un comportamiento más caótico y le cuesta más converger, aunque, a cambio, se ejecuta mucho más rápido, por lo que en el tiempo que el GD necesita para ejecutar una época, el SGD puede ejecutar miles.

**Como siempre, la virtud se encuentra en el término medio. Véis la linea verde, verdad? Esa es la buena! ¿Cómo llegamos a ella?**









Bueno, vamos a pensar qué tenemos hasta ahora. Tenemos:

* Un método que calcula las predicciones y errores de **todos** los elementos de nuestro training set: **(Vanilla) Gradient Descent**
* Un método que calcula las predicciones y errores de **1 elemento** escogido aleatoriamente de nuestro training set: **Stochastic Gradient Descent**

¿Qué os parecería que en vez de **1 solo elemento**, escogiésemos **N elementos**? De esta forma:

* Aumentamos la **estabilidad** del algoritmo, ya que no solo nos fijamos en un solo elemento, sino en $k$ (es decir, disminuímos los cambios de dirección tan abruptos y caóticos que tiene la línea magenta)
* Disminuimos el **tiempo de ejecución** con respecto al gradient descent tradicional, pues pasamos de las $N$ muestras que tiene nuestro training set, a $k$, donde $k << N$

Parece una alternativa interesante, no creéis? Este método se conoce como **Mini-batch Stochastic Gradient Descent**, y es realmente el que se utiliza en la práctica.

Normalmente, $k$ se elige para que sea una potencia de 2, ya que eso permite aprovechar algunas optimizaciones que tienen las GPUs implementadas para estos casos. Un $k$ típico podría ser $k=256$, pero al final esto lo limita la memoria de la GPU.

Cuanto más bajo sea $k$, más se parecerá al SGD puro, y más épocas le costará convergir, aunque también es verdad que las calculará más rápido.

Y a la inversa, cuanto más alto sea $k$, más se parecerá al GD puro, y más le costará calcular cada época, pero necesitará menos para convergir.

### Pues con esto ya lo sabéis todo sobre el Gradient Descent y los diferentes tipos de implementaciones!

Vamos a ver ahora qué es el learning rate y el batch size, y cómo influyen.


## 3.2 Learning rate y batch size

El learning rate y el batch size son dos parámetros directamente relacionados con el algoritmo del gradient descent.

### 3.2.1 Learning rate

Como ya sabéis de la anterior sesión, la forma de actualizar los pesos es mediante estas fórmulas:

<center><img src="https://image.ibb.co/jMmHRT/net_weights_update.png" alt="net_weights_update" border="0" height="250"></center>

¿Os acordáis, verdad? Lo que multiplica al $\frac{\partial E_{total}}{\partial w_n}$, llamado $\eta$, es el **learning rate**, que es lo que indica **la importancia** que le damos al error para actualizar cada peso, es decir, la **rapidez** o cómo de abruptos son los cambios en los pesos.

Así, un $\eta$ muy alto, hará que los cambios en los pesos sean muy grandes de una iteración a otra, lo que tiene el problema de que podemos llegar a saltarnos nuestro mínimo.

Fijaos, con esta imagen se ve estupendamente:

<img src="https://image.ibb.co/ncnAY8/learning_rate_eta.png" alt="learning_rate_eta" border="0" height="150">

Otra posibilidad es establecer un $\eta$ muy bajo, lo que haría que nuestra red necesitara muchísimas épocas para llegar a un mínimo aceptable. Además, correríamos el riesgo de quedarnos atrapados en un mínimo peor del mejor que podríamos conseguir con un $\eta$ más alto.

<img src="https://image.ibb.co/frt3Lo/learning_rate_eta_low.gif" alt="learning_rate_eta_low" border="0" height="200">

Vamos a hacer un pequeño inciso para hablar sobre los mínimos: lo que conseguimos con una **red neuronal**, normalmente, no es alcanzar el mínimo global de nuestra función, sino que **alcanzamos un mínimo local lo suficientemente bueno como para realizar correctamente la tarea que estamos desarrollando**.

Tras haber aclarado esto, queda patente lo importante que es conseguir un learning rate adecuado, verdad? Y cómo lo hacemos? Primero, qué es lo que buscamos? Fijaos en esta imagen:

<img src="https://image.ibb.co/heYQY8/learning_rate_eta_graph.png" alt="learning_rate_eta_graph" border="0" height="250">

Lo que queremos es un *learning rate* óptimo, que nos permita ir reduciendo el error conforme van pasando las épocas, hasta llegar a nuestro mínimo buscado. En la gráfica, este *learning rate* sería la línea roja. ¿Y cómo conseguimos que nuestro learning rate sea el óptimo?

Pues una opción muy utilizada es aplicar un **decrecimiento** o *decay* a nuestro learning rate **conforme más va disminuyendo la función de pérdidas** (lo que indica que estamos llegando al mínimo buscado).

<img src="https://image.ibb.co/mdBUt8/learning_rate_eta_decreasing.png" alt="learning_rate_eta_decreasing" border="0" height="250">

De esta forma, evitamos morirnos de viejos esperando a que converja por haber elegido un learning rate muy bajito, y evitamos saltarnos nuestro mínimo porque cuanto más cerca estamos de él más pequeños son los pasos que damos hacia él.

Vamos a ver cómo se comportaría la última red que implementamos (la que trabajaba con el MNIST) con **diferentes learning rates**:

Los valores típicos de learning rate suelen ser: 

* 0.1
* 0.01
* 0.001

A partir de ahí, nos toca a nosotros *tunear* el learning rate según lo que veamos que ocurre.

In [0]:
# ejemplos learning rate

Bueno, pues la mejor ejecución la hemos conseguido con learning rate = 0.1. Pero... ¿se puede mejorar?

Como podréis imaginaros, el learning rate es un parámetro muy importante. Ya hemos visto antes que si ponemos un learning rate muy alto, podemos saltarnos el mínimo. En cambio, si ponemos un learning rate muy bajo, igual nos morimos de viejos antes de ver el resultado. ¿Cómo podemos arreglar esto? Pues una forma sería dar pasos más grandes al principio, y conforme nos aproximamos a la meta, empezar a disminuir su longitud para no pasárnosla, no os parece?

Pues esto mismo se llama learning rate *decay*. ¿Qué os parece si lo implementamos en nuestro ejemplo?

Vamos allá!

In [0]:
# ejemplo con decay

Fijaos que con esta simple mejora, hemos sido capaces de mejorar la precisión un poco con respecto al learning rate óptimo, que habíamos definido como 0.1. Y existen métodos más complejos de aplicar el *decay* que producen mejores resultados.

**Así que ya sabéis, mucha atención al learning rate!!**

**IMPORTANTE**: Cabe decir que esta implementación de learning rate *decay* es muy básica, tanto, que se cambia el learning rate para cada batch dentro de una época. Esto no es lo mejor, pero lo hemos hecho así por no complicar la programación. **Lo adecuado sería que se actualizara el learning rate para cada época**, no para cada batch. Tranquiiiilos, lo veremos en la siguiente sesión! ;-)


### 3.2.2 Batch size

Vamos a ver ahora qué es el **batch size**. ¿Recordáis cuando hemos hablado antes del SGD y el Mini-batch SGD? 

Recordad que el SGD es un Mini-bacth SGD donde $k=1$.

Y que en el Mini-bacth SGD la $k$ indica el número de muestras que se utilizan para actualizar los pesos cada vez. Realmente, este no es un parámetro crítico y se suele establecer como el **número máximo de muestras potencia de 2 que caben en nuestra GPU**.

Ejemplos:

* Tenemos una GPU con 8GB de memoria, cuantas muestras nos caben si cada imagen ocupa 1MB?

Bueno, pues no es tan fácil! Resulta que depende de la arquitectura de la red. Las capas *Densas* o *Fully Connected*, que son las tradicionales en las que todas las neuronas se interconectan con todas las neuronas de la siguiente capa, son las que **más parametros tienen**, y por consiguiente, las que **más memoria ocupan**.

Luego tambien tenemos capas convolucionales, de pooling, de dropout, y de muchos otros tamaños. Así que en la práctica, es difícil calcular a mano cual es el número máximo de muestras que podemos usar.

Lo que se hace es probar con tamaños de batch potencia de 2 e ir disminuyéndolo si tenemos un error de memoria. Por ejemplo, podríamos empezar con 512, y si nos da error ir bajando a 256, 128, 64, 32, 16, 8, 4, 2 e incluso 1. Dependiendo de la arquitectura de nuestra red, puede llegar a pasarnos que tengamos que usar $k=1$, y por tanto, SGD.

Aunque muchas veces es preferible disminuir el tamaño de la imagen, por ejemplo, de 512x512 a 256x256 o 128x128 pixels, y usar un $k$ mayor.

Yo por ejemplo he tenido que usar $k=1$ con imágenes de 512x512 y la arquitectura DenseNet. **Y no hay problema**, simplemente tarda **más tiempo** en llegar a una solución adecuada.

Vamos a ver algunos casos de diferentes bath size siguiendo con el ejemplo anterior:

In [0]:
# ejemplos batch size

Es muy importante tener en cuenta que el **learning rate va relacionado con el batch size**.

Si nos aproximamos a $k=1$, debemos bajar el learning rate para que las actualizaciones de los pesos tengan menos importancia, ya que cada vez se aproxima más al SGD, es decir, a cálculos del gradiente con muestras aleatorias. 

Veamos qué pasa si disminuimos el learning rate:

In [0]:
# pruebas jugando con el learning rate y el batch size

Y si os fijáis en la curva de pérdidas, podéis intuir que todavía es demasiado grande, puesto que la función no es monótona decreciente, es decir, que no disminuye en cada época. Así que sería mejor usar un learning rate aun menor. Veámoslo:

## 3.3 Funciones de pérdidas y funciones de activación

### 3.3.1 Funciones de pérdidas

¿Qué es una función de pérdidas (o de coste)? ¿Quién sabe decírmelo?

La función de pérdidas es la que nos indica en cuánto nos hemos equivocado con nuestras predicciones.

Imaginad que tenemos que adivinar cuánto cuesta una casa simplemente viendo una foto. Nuestra red tendría como entrada los píxels de la foto, y como salida un número indicando el precio.

Por ejemplo, digamos que queremos predecir el precio de esta casa:

<img src="https://cdn.elgrupoinformatico.com/img/w720/Noticias/2018/05/memes-chalet-pablo-iglesias-irene-montero-720x362.jpg" border="0">

Imaginaos que estamos entrenando la red y que esta casa está dentro de nuestro conjunto de training. Cuando la imagen pasa hacia delante, se calcula una predicción, que es que vale 323.567€. La verdad es que la casa cuesta 600.000€, así que parece obvio que una función de pérdidas adecuada podría ser:

$f_{loss} = prediccion - valor\_real$

¿Lo entendéis verdad? Es exactamente lo mismo que con un SVM o la regresión lineal, por ejemplo.

Bueno, pues para no enrollarme mucho, estas son las funciones de pérdidas más comunes:

* **Problemas de regresión**
 * Mean Squared Error
 * Mean Absolute Error
* **Problemas de clasificación**
 * Binary Cross-Entropy
 * Categorical Cross-Entropy

Veamos qué son cada una:

* Mean Squared Error

$$MSE = \frac{1}{n} \sum^{n}_{i=1} (y_i-x_i)^2$$

* Mean Absolute Error

$$MAE = \frac{1}{n} \sum^{n}_{i=1} |y_i-x_i|$$

Hasta aquí todo bien verdad? Pero.. qué es la entropía cruzada o cross-entropy?

Vamos a verlo en el notebook de la Cros-entropía.

Bueno, pues ahora que está claro qué son las funciones de pérdidas, vamos a por las de activación!

### 3.3.2 Funciones de activación

Os he de confesar una cosa: la magia de las redes neuronales no es solo el back-propagation. Sin las funciones de activación, las redes neuronales no funcionarían. ¿Alguien sabe decirme por qué?

..


..


..

Os doy una pista:

### Qué pasaría si no existiese la función de activación?

<img src="https://image.ibb.co/ftYCVo/what_if_no_activation_function.png" alt="what_if_no_activation_function" border="0" width="600">

Tendríamos que $y(x)= Wx + b$. Esto es una combinación lineal que sería incapaz incluso de resolver un problema como el XOR.

<img src="https://image.ibb.co/kg9GO8/xor.gif" alt="xor" border="0">

Por lo tanto, necesitamos una forma de introducir **no linealidades**, y de eso es de lo que se encarga la **función de activación**. En la siguiente imagen podéis ver algunas de las más típicas, y donde interviene en la red:

<img src="https://image.ibb.co/bVS6qo/perceptron_activation.png" alt="perceptron_activation" border="0" width="600">

Aquí podéis ver las más usadas:

<img src="https://image.ibb.co/gMG5kd/activation_functions.png" alt="activation_functions" border="0" width="600">

Es difícil saber con cual de ellas nuestra red se va a comportar mejor, pero existe una que suele dar buenos resultados casi siempre: la **ReLU**. Por lo tanto, siempre que empecemos, emplearemos la ReLU, y una vez consigamos unos resultados que consideremos buenos, podemos probar con la Leaky ReLU, o cualquier otra que os apetezca. Cada día salen nuevas, y una simple búsqueda en google os puede llevar a alguna interesante: como la SELU, por ejemplo (https://towardsdatascience.com/selu-make-fnns-great-again-snn-8d61526802a9).

Muchas de estas funciones de activación necesitan métodos específicos de **inicialización de pesos**, para que esten dentro de unos valores y que el descenso del gradiente funcione adecuadamente.

Veamos qué es lo que pasa si empleamos diferentes funciones de activación con nuestra ya conocida red:

In [0]:
# ejemplos de funciones de activación

# Vamos a probar con la Sigmoide

<img src="https://image.ibb.co/cMMqJ8/act_fn_sigmoid.png" alt="act_fn_sigmoid" border="0" height="250">

In [0]:
# probamos la sigmoide

# Veamos con la tanh

<img src="https://image.ibb.co/haCKBT/act_fn_tanh.png" alt="act_fn_tanh" border="0" height="250">

In [0]:
# probamos la tanh

### ¡Vaya! ¿Qué está pasando?

Probemos a cambiar la inicialización de la bias, en vez de a 0s, a 1s.

In [0]:
# probamos la tanh y bias 1 en vez de 0

### Parece que hemos conseguido mejorarlo!!

** Ahora, por lo menos, nuestra red se entrena... pero no se comporta todo lo bien que esperábamos, verdad?**

Vamos a probar a cambiar la función de pérdidas, de la cros-entropía al error cuadrático medio:

In [0]:
# probamos la tanh, bias = 1 y loss_fn = mse

¿Os dáis cuenta? Hay 3 cosas que van íntimamente ligadas y de las que debemos ocuparnos para encontrar una buena configuración:

* **La función de activación**
* **La función de pérdidas**
* **La inicialización de los pesos y bias**

In [0]:
# probamos la tanh, weights = 1, bias = 1 y loss_fn = mse

### Y si inicializamos los pesos aleatoriamente escogiendolos de una distribución normal *truncada*?

In [0]:
# probamos la tanh, weights = 1, bias = 1 y loss_fn = mse

### Lo que pretendo con estos ejemplos es que os deis cuenta de la importancia de estos 3 parámetros. Recordad, merece la pena *perder* un poco de tiempo hasta encontrar una configuración adecuada de:

* **La función de activación**
* **La función de pérdidas**
* **La inicialización de los pesos y bias**

### También es importante que tengáis en cuenta que ahora mismo, nuestra red tiene una sola capa. Cuando tengamos redes con varias capas ocultas, hay funciones de activación que se deben utilizar en las capas ocultas y otras en las de salida. Por ejemplo, las de tipo ReLU, ELU y Maxout, son para las capas ocultas. La softmax es la que se suele emplear siempre en la salida, ya que da una distribución de probabilidades que suman 1.

Vamos a seguir con las otras funciones de activación. Usaremos la red que usa el MSE y una bias = 1.

# Veamos con la ReLU

<img src="https://image.ibb.co/mSsKBT/act_fn_relu.png" alt="act_fn_relu" border="0" height="250">

<img src="https://image.ibb.co/iUdsWT/act_fn_relu2.png" alt="act_fn_relu2" border="0" height="250">

<img src="https://image.ibb.co/fzXako/act_fn_relu3.png" alt="act_fn_relu3" border="0" height="250">

In [0]:
# probamos la ReLU

# Veamos con la Leaky RELU

<img src="https://image.ibb.co/j5pAJ8/act_fn_lrelu.png" alt="act_fn_lrelu" border="0" height="250">

In [0]:
# probamos la Leaky ReLU, bias = 1 y loss_fn = mse

# Veamos con la ELU

<img src="https://image.ibb.co/gEJgQo/act_fn_elu.png" alt="act_fn_elu" border="0" height="250">

In [0]:
# probamos la ELU, bias = 1 y loss_fn = mse

# Veamos con la Maxout

<img src="https://image.ibb.co/hWg1Qo/act_fn_maxout.png" alt="act_fn_maxout" border="0" height="250">

Básicamente esta función de activación devuelve el máximo de múltiples posibles salidas para cada entrada. Al final, es una generalización de la ReLU y la Leaky ReLU. Para la ReLU, por ejemplo, tenemos $w_1=b_1=0$). Por lo tanto, la Maxout se beneficia de todas las ventajas de la ReLU (es lineal y no satura) y además evita matar el gradiente cuando $x<0$.

Para que nos entendamos, podemos aproximar cualquier función con varias maxouts, como podéis ver en la imagen siguiente, en la que aproximamos $y=x^2$ con 3 maxouts:

<img src="https://image.ibb.co/co951T/maxout_aproximator.png" alt="maxout_aproximator" border="0" height="250">

Este tipo de activación funciona bien cuando se combina con Dropout, que lo veremos al final de esta sesión.

No os preocupéis, más adelante implementaremos nuestra función de activación Maxout! ;-)

Si queréis saber más, aquí tenéis una presentación de Ian Goodfellow donde explican cómo funciona:  http://www.ms.uky.edu/~qye/MA721/maxout.pdf

Y aquí el paper original: https://arxiv.org/abs/1302.4389

Más info: 

 * http://cs231n.github.io/neural-networks-1/

# Atención:

### Como he comentado antes, en el caso de las capas de salida, practicamente siempre se utiliza la función de activación de tipo softmax, ya que es capaz de dar una probabilidad a cada clase, haciendo que todas ellas sumen 1.

Como esto puede parecer un poco complicado, os voy a escribir aquí la receta que sigo yo y que hasta ahora me ha dado buenos resultados:

# Receta

* Empezar usando la ReLU con un learning rate de 0.01 o 0.001, y observar qué pasa.

* Si la red entrena (va convergiendo) pero es lenta, podéis probar a aumentar un poco el learning rate

* Si la red no converge y se compora de forma caótica, disminuid el learning rate

* Una vez tengáis la red funcionando, probad con la Leaky ReLU, la Maxout o la ELU

* No uséis la sigmoide, en la práctica no suele dar buenos resultados


## 3.5 Inicialización de pesos

Como habéis visto antes, la inicialización de los pesos y la bias es muy importante para conseguir la convergencia de nuestra red a un mínimo adecuado. Así que vamos a ver algunas formas de inicializar los pesos.

Siguiendo con nuestro ejemplo del MNIST, nuestra matriz de pesos va a ser de 768 (entradas) x 10 (salidas).

### 3.5.1 Inicialización constante

Podemos inicializar nuestros pesos a cero:

`W = np.zeros((768, 10))`

A uno:

`W = np.ones((768, 10))`

O a una constante $C$:

`W = np.ones((768, 10)) * C`

### 3.5.2 Distribución normal y uniforme

También podemos inicializar los pesos usando una distribución **uniforme**, en la que se definen un $[upper\_bound, lower\_bound]$ y todos los números dentro del rango tienen la misma probabilidad de ser escogidos. Por ejemplo, para una distribución entre $[-0.2, 0.2]$:

`W = np.random.uniform(low=-0.2, high=0.2, size=(768, 10))`

Con esta instrucción, inicializaremos la matriz de pesos $W$ con valores extraídos del rango entre $[-0.2, 0.2]$ donde todos ellos tienen la misma probabilidad de ser extraídos.

También podemos hacerlo con una distribución **normal** o Gaussiana, la cual viene definida como:

$$p(x)=\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

Donde como ya sabéis:

* $\mu$ es la media
* $\sigma$ es la desviación estándar, y $\sigma^2$ la varianza

Así que podríamos inicializar nuestros pesos con una distribución normal con $\mu = 0$ y $\sigma = 0.2$, por ejemplo, de la siguiente forma:

`W = np.random.normal(0.0, 0.2, size=(768, 10))`

### 3.5.3 Inicialización: LeCun normal y uniforme

Otra forma más avanzada es el método de LeCun, también conocido como *"Efficient backprop"*.

Este método define 3 parámetros:

* $f_{in}$: número de entradas a la capa (en nuestro ejemplo, 768)
* $f_{out}$: número de salidas de la capa (en nuestro ejemplo, 10)
* $limit$: queda definido según $f_{in}$ y $f{out}$ como $limit = \sqrt{\frac{3}{f_{in}}}$

El código para inicializar $W$ mediante este método usando una distribución uniforme sería:

`W = np.random.uniform(low=-limit, high=limit, size=(F_in, F_out))`

Y para una normal:

`W = np.random.normal(low=-limit, high=limit, size=(F_in, F_out))`

### 3.5.4 Inicialización: Glorot/Xavier normal y uniforme

Esta es quizás el método más empleado a la hora de inicializar los pesos y la bias. De hecho, es el empleado por defecto cuando se utiliza Keras, un framework que veréis en la próxima sesión.

En este caso también se definen los mismos parámetros que con LeCun, pero varía el cálculo del $limit$:

 $limit = \sqrt{\frac{2}{f_{in}+f_{out}}}$
 
 El código para inicializar $W$ mediante este método sería el mismo que con LeCun. 
 
 Para una distribución uniforme sería:

`W = np.random.uniform(low=-limit, high=limit, size=(F_in, F_out))`

Y para una normal:

`W = np.random.normal(low=-limit, high=limit, size=(F_in, F_out))`

### 3.5.5 Inicialización: He et al./Kaiming/MSRA normal y uniforme

Este método debe su nombre a Kaiming He, el primer autor de *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*.

Normalmente, este método se usa cuando estamos entrenando redes neuronales *muy profundas* que usan un tipo particular de ReLU como activación: la Parametric ReLU).

El código en el caso de la uniforme es este:

`limit = np.sqrt(6 / float(F_ini))`

`W = np.random.uniform(low=-limit, high=limit, size=(F_in, F_out))`


Y en el caso de la normal, este:


`limit = np.sqrt(2 / float(F_ini))`

`W = np.random.uniform(low=-limit, high=limit, size=(F_in, F_out))`


## Consejo:

La inicialización de los pesos no suele ser determinante en el entrenamiento de una red, pero a veces puede hacer que la red no empiece a entrenar porque no consigue converger. Por lo tanto, mi consejo es que uséis la de Glorot, y si ese día estáis aventureros y queréis ver si conseguís hacer que mejore la precisión, probéis con otras ;-)

